In [3]:
!export TF_CPP_MIN_LOG_LEVEL=2 TF_CPP_MIN_LOG_LEVEL=2

In [4]:
import pickle

import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

from tensorflow import keras
import kerasncp as kncp

import os
from typing import Iterable, Dict
import tensorflow as tf
import kerasncp as kncp
from kerasncp.tf import LTCCell, WiredCfcCell
from tensorflow import keras
import numpy as np
from matplotlib.image import imread
from tqdm import tqdm
from PIL import Image
import pandas as pd
import time
from keras_models import generate_ncp_model
from train_test_loader import get_dataset_multi, get_val_dataset_multi

def tlen(dataset):
    for (ix, _) in enumerate(dataset):
        pass
    return ix

training_root = "../quadrant_wise_dataset/mix_goal_heights_diff"
# val_root = "../fly_to_target_dataset/test_data"
DROPOUT = 0.1

DEFAULT_NCP_SEED = 22222

IMAGE_SHAPE = (144, 256, 3)
IMAGE_SHAPE_CV = (IMAGE_SHAPE[1], IMAGE_SHAPE[0])

batch_size = None
seq_len = 64
augmentation_params = None
single_step = False
no_norm_layer = False

decay_rate: float = 0.85
lr: float = 0.001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=lr, decay_steps=500,
                                                            decay_rate=decay_rate, staircase=True)
#Adam optimizer
optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)

gpus = tf.config.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy(gpus)
with strategy.scope():
    mymodel = generate_ncp_model(seq_len, IMAGE_SHAPE, augmentation_params, batch_size, DEFAULT_NCP_SEED, single_step, no_norm_layer)
    mymodel.compile(optimizer=optimizer, loss="mean_squared_error", metrics=['mse'])
    # mymodel.load_weights('saved_models/retrain_diff_goal_mix_3to5_wscheduler0.85_seed22222_lr0.001_trainloss0.00050_epoch100.h5')

    mymodel.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64, 144, 256, 3)] 0         
_________________________________________________________________
rescaling_1 (Rescaling)      (None, 64, 144, 256, 3)   0         
_________________________________________________________________
time_distributed_9 (TimeDis

In [8]:

def print_model_summary_with_activations(model):
    """Prints the model summary along with activation functions"""
    print("\nModel Summary with Activation Functions:\n")
    print(f"{'Layer Name':<30}{'Layer Type':<25}{'Activation Function':<20}{'Param #':<10}")
    print("=" * 100)

    for layer in model.layers:
        # Check if it's a TimeDistributed layer
        if isinstance(layer, keras.layers.TimeDistributed):
            wrapped_layer = layer.layer  # Extract the inner layer
            activation = getattr(wrapped_layer, "activation", None)
            activation_name = activation.__name__ if activation else "-"
            layer_type = f"TimeDistributed({wrapped_layer.__class__.__name__})"
        else:
            activation = getattr(layer, "activation", None)
            activation_name = activation.__name__ if activation else "-"
            layer_type = layer.__class__.__name__

        print(f"{layer.name:<30}{layer_type:<30}{activation_name:<20}{layer.count_params():<10}")

    print("=" * 100)
    print(f"Total Parameters: {model.count_params()}")




In [9]:
print_model_summary_with_activations(mymodel)


Model Summary with Activation Functions:

Layer Name                    Layer Type               Activation Function Param #   
input_2                       InputLayer                    -                   0         
rescaling_1                   Rescaling                     -                   0         
time_distributed_9            TimeDistributed(Normalization)-                   7         
time_distributed_10           TimeDistributed(Conv2D)       relu                1824      
time_distributed_11           TimeDistributed(Conv2D)       relu                21636     
time_distributed_12           TimeDistributed(Conv2D)       relu                43248     
time_distributed_13           TimeDistributed(Conv2D)       relu                27712     
time_distributed_14           TimeDistributed(Conv2D)       relu                9232      
time_distributed_15           TimeDistributed(Flatten)      -                   0         
time_distributed_16           TimeDistributed(Dense)